In [ ]:
import numpy as np
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline


import strax

In [ ]:
records = strax.load('tl_z')
strax.process.baseline(records)

In [ ]:
from pax.configuration import load_configuration
from pax.dsputils import adc_to_pe
pax_config = load_configuration('XENON1T')["DEFAULT"]
#gains = np.array(pax_config['DEFAULT']['gains'])

In [ ]:
to_pe = np.array([adc_to_pe(pax_config, ch) for ch in range(pax_config['n_channels'])])

In [ ]:
# import numba

# @numba.jit(nopython=True)
# def to_pe(adc, channel, gains):
#     output = np.zeros(len(adc), dtype=np.float32)
    
#     for i in range(len(adc)):
#         output[i] = adc * ch
    

In [ ]:
times = records['time']
areas = records['data'].sum(axis=1) * to_pe[records['channel']]

In [ ]:
import holoviews as hv
from holoviews.operation.datashader import aggregate, shade, datashade, dynspread
from holoviews.operation import decimate
hv.extension('bokeh')

In [ ]:
loga = np.ones_like(areas) * -1
loga[areas > 0] = np.log10(areas[areas > 0])
loga[~(loga > -1)] = -1

In [ ]:
t = (times - times[0])/1e9

In [ ]:
df = pd.DataFrame(dict(log_area=loga, time=t, channel=records['channel']))

In [ ]:
points = hv.Points(df, kdims=['time', 
                             hv.Dimension('channel', label='PMT number', range=(0, 260))
                             ], vdims=['log_area'])
points2 = points.clone()

In [ ]:
# Log axis:
#.opts(plot=dict(logy=True))

In [ ]:
# Unspeakably evil hack to disable default tools (which I can't customize)
hv.plotting.bokeh.element.ElementPlot.default_tools = 'save pan box_zoom reset'.split()

# Custom wheel zoom tool that only zooms in time
from bokeh.models import WheelZoomTool
wzt = WheelZoomTool(dimensions='width')

In [ ]:
points.dimensions

In [ ]:
import datashader as ds

In [ ]:
agg = aggregate(points2, 
                aggregator=ds.count(), 
                streams=[hv.streams.RangeX()], 
                #x_sampling=0.01, 
                width=900,
                height=2)
waveform = agg.map(lambda x: x.reduce(channel=np.sum), hv.Image)
waveform

In [ ]:
%%opts RGB [width=900, tools=[wzt]]
%%opts Curve [width=900, tools=[wzt]] 

agg = aggregate(points2, 
                aggregator=ds.count(), 
                streams=[hv.streams.RangeX(source=points)], 
                #x_sampling=0.01, 
                width=900,
                height=2)
waveform = agg.map(lambda x: x.reduce(channel=np.sum), hv.Image).opts(plot=dict(logy=True))

channelmap = datashade(points,
                       streams=[hv.streams.RangeX(source=points2)], 
                       y_range=(0, 260))

(waveform + channelmap).cols(1)

In [ ]:
bla[1].default_tools